In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [28]:
import pandas as pd
import sqlite3
import os
import shutil
from tqdm import tqdm
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
import matplotlib.pyplot as plt

In [3]:
fname = "/home/boris/Data/SIFTS/pdb_chain_uniprot.csv"
sifts_df = pd.read_csv(fname, comment='#')
sifts_df

,PDB,CHAIN,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END
0,101m,A,P02185,1,154,0,153,1,154
1,102l,A,P00720,1,40,1,40,1,40
2,102l,A,P00720,42,165,41,None,41,164
3,102m,A,P02185,1,154,0,153,1,154
4,103l,A,P00720,1,40,1,None,1,40
...,...,...,...,...,...,...,...,...,...
635822,9xia,A,P24300,1,388,1,None,1,388
635823,9xim,A,P12851,1,393,None,394,2,394
635824,9xim,B,P12851,1,393,None,394,2,394
635825,9xim,C,P12851,1,393,None,394,2,394


In [4]:
con = sqlite3.connect("/home/boris/Data/ChemBL/chembl_30/chembl_30_sqlite/chembl_30.db")

In [5]:
cursor = con.cursor()
cursor.execute("create table if not exists sifts (pdb text, chain text sp_primary text, res_beg integer, res_end integer, pdb_beg integer, pdb_end integer, sp_beg integer, sp_end integer)")
cursor.fetchall()

sifts_df.to_sql('sifts', con, if_exists='replace', index=False)
pd.read_sql_query("select * from sifts", con)

,PDB,CHAIN,SP_PRIMARY,RES_BEG,RES_END,PDB_BEG,PDB_END,SP_BEG,SP_END
0,101m,A,P02185,1,154,0,153,1,154
1,102l,A,P00720,1,40,1,40,1,40
2,102l,A,P00720,42,165,41,None,41,164
3,102m,A,P02185,1,154,0,153,1,154
4,103l,A,P00720,1,40,1,None,1,40
...,...,...,...,...,...,...,...,...,...
635822,9xia,A,P24300,1,388,1,None,1,388
635823,9xim,A,P12851,1,393,None,394,2,394
635824,9xim,B,P12851,1,393,None,394,2,394
635825,9xim,C,P12851,1,393,None,394,2,394


In [11]:
rec_files_g = crossdocked['rec_file'].unique()

In [23]:
rec2file = {}
for rfg in rec_files_g:
    rf = "/home/boris/Data/CrossDocked/" + "_".join(rfg.split("_")[:-1]) + ".pdb"
    rec = rf.split("/")[-1].split(".")[0]
    rec2file[rec] = rf

In [34]:
for struct_rec in tqdm(struct_df["rec_file"]):
    folder = "/home/boris/Data/BigBind/" + struct_rec.split("/")[0]
    rec = "_".join(struct_rec.split("/")[-1].split("_")[:-1])
    rf = rec2file[rec]
    out_file = f"{folder}/{rec}.pdb"
    shutil.copyfile(rf, out_file)

100%|███████████████████████████████████████████████████████████████████████████████| 525151/525151 [10:11:40<00:00, 14.31it/s]


In [6]:
crossdocked_types = "/home/boris/Data/CrossDocked/types/it2_tt_v1.1_completeset_train0.types"
crossdocked = pd.read_csv(crossdocked_types, sep=' ', names=["label", "binding_affinity", "crystal_rmsd", "rec_file", "lig_file", "vina_score"])

In [7]:
cd_pdbs = crossdocked['rec_file'].apply(lambda f: f.split('/')[-1].split('_')[0])
cd_chains = crossdocked['rec_file'].apply(lambda f: f.split('/')[-1].split('_')[1])

In [8]:
cd_pdbs = set(cd_pdbs)
cd_chains = set(cd_chains)

In [9]:
cd_chains_str = ", ".join(map(lambda s: f"'{s}'", cd_chains))
cd_pdbs_str = ", ".join(map(lambda s: f"'{s}'", cd_pdbs))

crossdocked_uniprots = pd.read_sql_query(f"select SP_PRIMARY from sifts where PDB in ({cd_pdbs_str})", con)
cd_uniprots_str = ", ".join(map(lambda s: f"'{s}'", crossdocked_uniprots["SP_PRIMARY"]))

In [17]:
%%time
query =   f"""

SELECT md.chembl_id AS compound_chembl_id,
cs.canonical_smiles,
act.standard_type,
act.standard_relation,
act.standard_value,
act.standard_units,
act.pchembl_value,
a.confidence_score,
td.chembl_id AS target_chembl_id,
td.target_type,
c.accession as protein_accession,
a.chembl_id as assay_id
FROM target_dictionary td
  JOIN assays a ON td.tid = a.tid
  JOIN activities act ON a.assay_id = act.assay_id
  JOIN molecule_dictionary md ON act.molregno = md.molregno
  JOIN compound_structures cs ON md.molregno = cs.molregno
  JOIN target_type tt ON td.target_type = tt.target_type
  JOIN target_components tc ON td.tid = tc.tid
  JOIN component_sequences c ON tc.component_id = c.component_id
  AND tt.parent_type  = 'PROTEIN' 
  AND act.potential_duplicate = 0
  AND a.confidence_score >= 8
  AND act.data_validity_comment IS NULL
  AND act.pchembl_value IS NOT NULL
  AND c.accession in ({cd_uniprots_str});
  
"""
filename = "/home/boris/Data/BigBind/activities.csv"
with open(filename, 'w'): pass

chunks = pd.read_sql_query(query,con,chunksize=1000)
header = True
for i, chunk in enumerate(tqdm(chunks)):

    chunk.to_csv(filename, header=header, mode='a', index=False)

    header = False
    # if i > 0:
    #     break

chunk

1280it [41:48,  1.96s/it]

CPU times: user 1min 53s, sys: 58.3 s, total: 2min 51s
Wall time: 41min 48s


,compound_chembl_id,canonical_smiles,standard_type,standard_relation,standard_value,standard_units,pchembl_value,confidence_score,target_chembl_id,target_type,protein_accession,assay_id
0,CHEMBL4787602,Fc1cc(Br)cc(CNc2ccc3[nH]nc(/C=C/c4ccccn4)c3c2)c1,IC50,=,30.6,nM,7.51,9,CHEMBL4898,SINGLE PROTEIN,Q16620,CHEMBL4772613
1,CHEMBL4796220,Cc1cc(F)cc(CNc2ccc3[nH]nc(/C=C/c4ccccn4)c3c2)c1,IC50,=,50.1,nM,7.30,9,CHEMBL4898,SINGLE PROTEIN,Q16620,CHEMBL4772613
2,CHEMBL4800232,COc1cc(F)cc(CNc2ccc3[nH]nc(/C=C/c4ccccn4)c3c2)c1,IC50,=,26.7,nM,7.57,9,CHEMBL4898,SINGLE PROTEIN,Q16620,CHEMBL4772613
3,CHEMBL4794286,Fc1cc(CNc2ccc3[nH]nc(/C=C/c4ccccn4)c3c2)cc(C(F...,IC50,=,245.2,nM,6.61,9,CHEMBL4898,SINGLE PROTEIN,Q16620,CHEMBL4772613
4,CHEMBL4783132,N#Cc1cc(F)cc(CNc2ccc3[nH]nc(/C=C/c4ccccn4)c3c2)c1,IC50,=,17.7,nM,7.75,9,CHEMBL4898,SINGLE PROTEIN,Q16620,CHEMBL4772613
...,...,...,...,...,...,...,...,...,...,...,...,...
868,CHEMBL4781656,COc1ccc(S(=O)(=O)c2cc(C(N)=O)c(NC(=O)C(C)(C)O)...,EC50,=,478.0,nM,6.32,9,CHEMBL4051,SINGLE PROTEIN,P13569,CHEMBL4775871
869,CHEMBL4785210,C[C@H](O)CNC(=O)c1ncc(S(=O)(=O)c2ccc(OC(F)(F)F...,EC50,=,18.0,nM,7.75,9,CHEMBL4051,SINGLE PROTEIN,P13569,CHEMBL4775883
870,CHEMBL4785210,C[C@H](O)CNC(=O)c1ncc(S(=O)(=O)c2ccc(OC(F)(F)F...,EC50,=,26000.0,nM,4.58,9,CHEMBL3397,SINGLE PROTEIN,P11712,CHEMBL4775889
871,CHEMBL12089,COc1ccc2cc3[n+](cc2c1OC)CCc1cc2c(cc1-3)OCO2.[Cl-],IC50,=,4200.0,nM,5.38,9,CHEMBL4849,SINGLE PROTEIN,Q8TDX7,CHEMBL4775923


In [ ]:
vars = []
cvars = []
dfs = df.sample(frac=1)
chembl_values = 10**(9-dfs["pchembl_value"])
for cid in tqdm(dfs["compound_chembl_id"].unique()):
    idxs = dfs["compound_chembl_id"] == cid
    rows = dfs[idxs]
    if len(rows) == 1: continue
    vars.append(rows["pchembl_value"].var())
    cvars.append(chembl_values[idxs].var())

In [ ]:
chembl_values.std()

In [ ]:
np.sqrt(np.array(cvars)).mean()

In [ ]:
print(np.array(vars).mean()/df["pchembl_value"].var())
print(np.array(cvars).mean()/chembl_values.var())

In [ ]:
print(np.sqrt(np.array(vars).mean())/df["pchembl_value"].std())
print(np.sqrt(np.array(vars).mean())/chembl_values.std())

In [ ]:
con.close()

In [4]:
from collections import defaultdict
import requests
from traceback import print_exc
from bigbind import *

In [22]:
df = crossdocked# get_crossdocked_df()

In [23]:
chain2uniprot = {}
for i, row in tqdm(sifts_df.iterrows()):
    chain2uniprot[(row["PDB"], row["CHAIN"])] = row["SP_PRIMARY"]

635827it [00:41, 15369.19it/s]


In [24]:
uniprot2recs = defaultdict(set)
uniprot2ligs = defaultdict(set)
uniprot2pockets = defaultdict(set)
pocket2recs = defaultdict(set)
pocket2ligs = defaultdict(set)
for lig_file_g in tqdm(df["lig_file"]):
    dir = "/home/boris/Data/CrossDocked/"
    folder, file = lig_file_g.split("/")
    pocket = folder
    rec_id, rec_chain, _, lig_id, lig_name, *rest = file.split("_")
    rec_file = f"{dir}/{folder}/{rec_id}_{rec_chain}_rec.pdb"
    lig_file = f"{dir}/{folder}/{lig_id}_{lig_name}_lig.pdb"
    key = (rec_id, rec_chain)
    if key in chain2uniprot:
        uniprot = chain2uniprot[key]
        uniprot2recs[uniprot].add(rec_file)
        uniprot2ligs[uniprot].add(lig_file)
        uniprot2pockets[uniprot].add(pocket)
    pocket2recs[pocket].add(rec_file)
    pocket2ligs[pocket].add(lig_file)

100%|██████████████████████████████████████████████████████████████████████████| 22583743/22583743 [00:48<00:00, 464523.14it/s]


In [ ]:
for uniprot in tqdm(uniprot2recs):
    ligs = uniprot2ligs[uniprot]
    for lig in ligs:
        try:
            out_file = f"{lig}_untrans.sdf"
            if os.path.isfile(out_file): continue
            url = get_lig_url(lig)
            res = requests.get(url)
            with open(out_file, "w") as f:
                f.write(res.text)
        except KeyboardInterrupt:
            raise
        except:
            pass

In [ ]:
mol = Chem.MolFromMolFile("/home/boris/Data/BigBind/Q9Y478/4qfr_32j_lig.mol2")
mol

In [ ]:
pdb_lig_df = pd.read_csv("/home/boris/Data/Components-smiles-stereo-oe.smi", header=None, sep="\t", names=["smiles", "id", "name"])
pdb_lig_df.set_index("id", inplace=True)
pdb_lig_df['smiles']['32J']

In [27]:
get_lig_url("/home/boris/Data/CrossDocked//AAKB1_HUMAN_70_159_0/4yef_4cq_lig.pdb")

'https://models.rcsb.org/v1/4yef/ligand?auth_seq_id=201&auth_asym_id=D&encoding=sdf&filename=lig.sdf'

In [ ]:
outputs = []

for uniprot in tqdm(uniprot2recs):
    try:
        recs = uniprot2recs[uniprot]
        ligs = uniprot2ligs[uniprot]
        outputs.append(get_pocket(uniprot, recs, ligs))
    except KeyboardInterrupt:
        raise
    except:
        raise
        print_exc()

In [ ]:
uniprot = "P00519"
recs = uniprot2recs[uniprot]
ligs = uniprot2ligs[uniprot]
output = get_pocket(uniprot, recs, ligs)

In [28]:
lig_files, rec_files, res_nums = output
for rec, res_num in zip(rec_files, res_nums):
    if "2g2h" in rec:
        print(rec, res_num)

/home/boris/Data/BigBind/P00519/2g2h_A_rec_pocket.pdb 107
/home/boris/Data/BigBind/P00519/2g2h_A_rec_pocket.pdb 0


In [37]:
uniprot2pocs = defaultdict(set)
for uniprot, recs in uniprot2recs.items():
    for rec in recs:
        poc = rec.split("/")[-2]
        uniprot2pocs[uniprot].add(poc)

In [40]:
single_pockets = { u for u in uniprot2pocs if len(uniprot2pocs[u]) == 1 }

In [57]:
with open("/home/boris/Downloads/lig.sdf", "w") as f:
    f.write(res.text)

In [15]:
struct_data = {
    "uniprot": [],
    "lig_file": [],
    "rec_file": [],
    "num_rec_residues": []
}
for output in outputs:
    if output is None: continue
    lig_files, rec_files, res_nums = output
    for rec, res_num in zip(rec_files, res_nums):
        for lig in lig_files:
            uniprot = lig.split("/")[-2]
            lig = "/".join(lig.split("/")[-2:])
            rec = "/".join(rec.split("/")[-2:])
            struct_data["uniprot"].append(uniprot)
            struct_data["lig_file"].append(lig)
            struct_data["rec_file"].append(rec)
            struct_data["num_rec_residues"].append(res_num)

In [16]:
struct_df = pd.DataFrame(struct_data)
struct_df

,uniprot,lig_file,rec_file,num_rec_residues
0,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1b0g_A_rec_pocket.pdb,14
1,P04439,P04439/2gj6_3ib_lig.sdf,P04439/2x4s_A_rec_pocket.pdb,10
2,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1t1w_A_rec_pocket.pdb,13
3,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1s9y_A_rec_pocket.pdb,14
4,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1jf1_A_rec_pocket.pdb,16
...,...,...,...,...
701159,P77173,P77173/1s1s_wac_lig.sdf,P77173/1f47_B_rec_pocket.pdb,50
701160,P77173,P77173/1s1j_iqz_lig.sdf,P77173/1s1s_B_rec_pocket.pdb,32
701161,P77173,P77173/1y2g_cl3_lig.sdf,P77173/1s1s_B_rec_pocket.pdb,32
701162,P77173,P77173/1y2f_wai_lig.sdf,P77173/1s1s_B_rec_pocket.pdb,32


In [13]:
struct_fname = "/home/boris/Data/BigBind/structures.csv"
struct_df = pd.read_csv(struct_fname)

In [43]:
print(len(struct_df))
struct_df = struct_df[struct_df["uniprot"].isin(single_pockets)]
print(len(struct_df))

701164
525151


In [ ]:
rdkit_loadable = {}
rdkit_embeddable = {}
for lig_file in tqdm(struct_df["lig_file"].unique()):
    file = f"/home/boris/Data/BigBind/{lig_file}"
    try:
        mol =  next(Chem.SDMolSupplier(file))
        if mol is None:
            loadable = False
            embeddable = False
        else:
            loadable = True
            id = AllChem.EmbedMolecule(mol)
            embeddable = (id != -1)
    except OSError:
        loadable = False
        embeddable = False
    rdkit_loadable[lig_file] = loadable
    rdkit_embeddable[lig_file] = embeddable

In [21]:
struct_df["lig_rdkit_loadable"] = np.zeros((len(struct_df),), dtype=bool)
struct_df["lig_rdkit_embeddable"] = np.zeros((len(struct_df),), dtype=bool)
rdkit_loadable_arr = []
rdkit_embeddable_arr = []
for i, lig_file in enumerate(tqdm(struct_df["lig_file"])):
    rdkit_loadable_arr.append(rdkit_loadable[lig_file])
    rdkit_embeddable_arr.append(rdkit_embeddable[lig_file])
    # struct_df["lig_rdkit_loadable"][i] = rdkit_loadable[lig_file]
    # struct_df["lig_rdkit_embeddable"][i] = rdkit_embeddable[lig_file]
struct_df["lig_rdkit_loadable"] = rdkit_loadable_arr
struct_df["lig_rdkit_embeddable"] = rdkit_embeddable_arr
struct_df

100%|█████████████████████████████████████████████████████████████████████████████| 701164/701164 [00:00<00:00, 1918686.15it/s]


,uniprot,lig_file,rec_file,num_rec_residues,lig_rdkit_loadable,lig_rdkit_embeddable
0,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1b0g_A_rec_pocket.pdb,14,True,True
1,P04439,P04439/2gj6_3ib_lig.sdf,P04439/2x4s_A_rec_pocket.pdb,10,True,True
2,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1t1w_A_rec_pocket.pdb,13,True,True
3,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1s9y_A_rec_pocket.pdb,14,True,True
4,P04439,P04439/2gj6_3ib_lig.sdf,P04439/1jf1_A_rec_pocket.pdb,16,True,True
...,...,...,...,...,...,...
701159,P77173,P77173/1s1s_wac_lig.sdf,P77173/1f47_B_rec_pocket.pdb,50,True,True
701160,P77173,P77173/1s1j_iqz_lig.sdf,P77173/1s1s_B_rec_pocket.pdb,32,True,True
701161,P77173,P77173/1y2g_cl3_lig.sdf,P77173/1s1s_B_rec_pocket.pdb,32,True,True
701162,P77173,P77173/1y2f_wai_lig.sdf,P77173/1s1s_B_rec_pocket.pdb,32,True,True


In [22]:
struct_df.query("lig_rdkit_loadable == True").count()

uniprot                 701164
lig_file                701164
rec_file                701164
num_rec_residues        701164
lig_rdkit_loadable      701164
lig_rdkit_embeddable    701164
dtype: int64

In [45]:
struct_fname = "/home/boris/Data/BigBind/structures.csv"
struct_df.to_csv(struct_fname, index=False)

In [23]:
activities = pd.read_csv("/home/boris/Data/BigBind/activities.csv")
activities

,compound_chembl_id,canonical_smiles,standard_type,standard_relation,standard_value,standard_units,pchembl_value,confidence_score,target_chembl_id,target_type,protein_accession,assay_id
0,CHEMBL1627453,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,420.0,nM,6.38,8,CHEMBL3181,SINGLE PROTEIN,P14061,CHEMBL615175
1,CHEMBL1627455,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,1000.0,nM,6.00,8,CHEMBL3181,SINGLE PROTEIN,P14061,CHEMBL615175
2,CHEMBL1627884,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,4300.0,nM,5.37,8,CHEMBL3181,SINGLE PROTEIN,P14061,CHEMBL615175
3,CHEMBL1627451,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,1150.0,nM,5.94,8,CHEMBL3181,SINGLE PROTEIN,P14061,CHEMBL615175
4,CHEMBL542453,C=CCN(C)CCCCCCOc1ccc(C(=O)c2ccc([N+](=O)[O-])c...,IC50,=,1.9,nM,8.72,8,CHEMBL3593,SINGLE PROTEIN,P48449,CHEMBL615200
...,...,...,...,...,...,...,...,...,...,...,...,...
1279868,CHEMBL4781656,COc1ccc(S(=O)(=O)c2cc(C(N)=O)c(NC(=O)C(C)(C)O)...,EC50,=,478.0,nM,6.32,9,CHEMBL4051,SINGLE PROTEIN,P13569,CHEMBL4775871
1279869,CHEMBL4785210,C[C@H](O)CNC(=O)c1ncc(S(=O)(=O)c2ccc(OC(F)(F)F...,EC50,=,18.0,nM,7.75,9,CHEMBL4051,SINGLE PROTEIN,P13569,CHEMBL4775883
1279870,CHEMBL4785210,C[C@H](O)CNC(=O)c1ncc(S(=O)(=O)c2ccc(OC(F)(F)F...,EC50,=,26000.0,nM,4.58,9,CHEMBL3397,SINGLE PROTEIN,P11712,CHEMBL4775889
1279871,CHEMBL12089,COc1ccc2cc3[n+](cc2c1OC)CCc1cc2c(cc1-3)OCO2.[Cl-],IC50,=,4200.0,nM,5.38,9,CHEMBL4849,SINGLE PROTEIN,Q8TDX7,CHEMBL4775923


In [8]:
rdkit_loadable = []
rdkit_embeddable = []
for smiles in tqdm(activities["canonical_smiles"]):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        loadable = False
        embeddable = False
    else:
        loadable = True
        id = AllChem.EmbedMolecule(mol)
        embeddable = (id != -1)
    rdkit_loadable.append(loadable)
    rdkit_embeddable.append(embeddable)

  0%|                                                                                 | 1063/1279873 [00:14<5:53:01, 60.37it/s][13:31:01] UFFTYPER: Warning: hybridization set to SP3 for atom 7
RDKit WARNING: [13:31:01] UFFTYPER: Warning: hybridization set to SP3 for atom 7
RDKit WARNING: [13:31:01] UFFTYPER: Warning: hybridization set to SP3 for atom 7
[13:31:01] UFFTYPER: Warning: hybridization set to SP3 for atom 7
RDKit ERROR: [13:31:01] UFFTYPER: Unrecognized charge state for atom: 12
[13:31:01] UFFTYPER: Unrecognized charge state for atom: 12
RDKit WARNING: [13:31:01] UFFTYPER: Warning: hybridization set to SP3 for atom 7
[13:31:01] UFFTYPER: Warning: hybridization set to SP3 for atom 7
  0%|                                                                                 | 1747/1279873 [00:25<4:30:55, 78.63it/s]RDKit ERROR: [13:31:11] UFFTYPER: Unrecognized charge state for atom: 1
[13:31:11] UFFTYPER: Unrecognized charge state for atom: 1
  0%|▏                                   

In [7]:
activities = pd.read_csv("/home/boris/Data/BigBind/activities.csv")
activities

,compound_chembl_id,canonical_smiles,standard_type,standard_relation,standard_value,standard_units,pchembl_value,confidence_score,target_chembl_id,target_type,protein_accession,assay_id,is_test
0,CHEMBL1627453,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,420.0,nM,6.38,8,CHEMBL3181,SINGLE PROTEIN,P14061,CHEMBL615175,False
1,CHEMBL1627455,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,1000.0,nM,6.00,8,CHEMBL3181,SINGLE PROTEIN,P14061,CHEMBL615175,False
2,CHEMBL1627884,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,4300.0,nM,5.37,8,CHEMBL3181,SINGLE PROTEIN,P14061,CHEMBL615175,False
3,CHEMBL1627451,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,1150.0,nM,5.94,8,CHEMBL3181,SINGLE PROTEIN,P14061,CHEMBL615175,False
4,CHEMBL542453,C=CCN(C)CCCCCCOc1ccc(C(=O)c2ccc([N+](=O)[O-])c...,IC50,=,1.9,nM,8.72,8,CHEMBL3593,SINGLE PROTEIN,P48449,CHEMBL615200,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279868,CHEMBL4781656,COc1ccc(S(=O)(=O)c2cc(C(N)=O)c(NC(=O)C(C)(C)O)...,EC50,=,478.0,nM,6.32,9,CHEMBL4051,SINGLE PROTEIN,P13569,CHEMBL4775871,False
1279869,CHEMBL4785210,C[C@H](O)CNC(=O)c1ncc(S(=O)(=O)c2ccc(OC(F)(F)F...,EC50,=,18.0,nM,7.75,9,CHEMBL4051,SINGLE PROTEIN,P13569,CHEMBL4775883,False
1279870,CHEMBL4785210,C[C@H](O)CNC(=O)c1ncc(S(=O)(=O)c2ccc(OC(F)(F)F...,EC50,=,26000.0,nM,4.58,9,CHEMBL3397,SINGLE PROTEIN,P11712,CHEMBL4775889,True
1279871,CHEMBL12089,COc1ccc2cc3[n+](cc2c1OC)CCc1cc2c(cc1-3)OCO2.[Cl-],IC50,=,4200.0,nM,5.38,9,CHEMBL4849,SINGLE PROTEIN,Q8TDX7,CHEMBL4775923,False


In [9]:
activities["rdkit_loadable"] = rdkit_loadable
activities["rdkit_embeddable"] = rdkit_embeddable
activities

,compound_chembl_id,canonical_smiles,standard_type,standard_relation,standard_value,standard_units,pchembl_value,confidence_score,target_chembl_id,target_type,protein_accession,assay_id,is_test,rdkit_loadable,rdkit_embeddable
0,CHEMBL1627453,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,420.0,nM,6.38,8,CHEMBL3181,SINGLE PROTEIN,P14061,CHEMBL615175,False,True,True
1,CHEMBL1627455,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,1000.0,nM,6.00,8,CHEMBL3181,SINGLE PROTEIN,P14061,CHEMBL615175,False,True,True
2,CHEMBL1627884,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,4300.0,nM,5.37,8,CHEMBL3181,SINGLE PROTEIN,P14061,CHEMBL615175,False,True,True
3,CHEMBL1627451,C[C@]12CC[C@@H]3c4ccc(O)cc4CC[C@H]3[C@@H]1C[C@...,IC50,=,1150.0,nM,5.94,8,CHEMBL3181,SINGLE PROTEIN,P14061,CHEMBL615175,False,True,True
4,CHEMBL542453,C=CCN(C)CCCCCCOc1ccc(C(=O)c2ccc([N+](=O)[O-])c...,IC50,=,1.9,nM,8.72,8,CHEMBL3593,SINGLE PROTEIN,P48449,CHEMBL615200,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279868,CHEMBL4781656,COc1ccc(S(=O)(=O)c2cc(C(N)=O)c(NC(=O)C(C)(C)O)...,EC50,=,478.0,nM,6.32,9,CHEMBL4051,SINGLE PROTEIN,P13569,CHEMBL4775871,False,True,True
1279869,CHEMBL4785210,C[C@H](O)CNC(=O)c1ncc(S(=O)(=O)c2ccc(OC(F)(F)F...,EC50,=,18.0,nM,7.75,9,CHEMBL4051,SINGLE PROTEIN,P13569,CHEMBL4775883,False,True,True
1279870,CHEMBL4785210,C[C@H](O)CNC(=O)c1ncc(S(=O)(=O)c2ccc(OC(F)(F)F...,EC50,=,26000.0,nM,4.58,9,CHEMBL3397,SINGLE PROTEIN,P11712,CHEMBL4775889,True,True,True
1279871,CHEMBL12089,COc1ccc2cc3[n+](cc2c1OC)CCc1cc2c(cc1-3)OCO2.[Cl-],IC50,=,4200.0,nM,5.38,9,CHEMBL4849,SINGLE PROTEIN,Q8TDX7,CHEMBL4775923,False,True,True


In [10]:
filename = "/home/boris/Data/BigBind/activities.csv"
activities.to_csv(filename, header=True, index=False)